In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd drive/MyDrive/CS\ 224N\ Project
%ls # verify that you are in the right directory

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1x7bmzM_qtbj3CPKzenuwvwocK9KiZzWU/CS 224N Project
'224N Experiments (GAN on 998 Samples).gsheet'
'224N Project Brainstorm.gdoc'
'224N Project Helpful Tutorials.gdoc'
'224N Project Milestone Notes.gdoc'
'224N Project Poster.gslides'
 Adversarial-T5_Structure_1.ipynb
 aita_clean.csv
 aita_comments.csv
 aita_test_set.csv
 aita_test_set.gsheet
 aita_train_set.csv
 aita_valid_set.csv
 banana/
 banana.ipynb
 bart-base-checkpoint-204000/
 bart-baseline-attempt2.ipynb
 bart-checkpoint-5000/
 Baseline_Text_Examples.gdraw
 bert-baseline.ipynb
 blueberry/
 blueberry.ipynb
 checkpoint.txt
 config.json
 csvs/
 dataset_agg.ipynb
 drive/
 Evaluate.ipynb
'experimenting with gumbel.ipynb'
 finetune-gpt2/
'first proposal OLD'/
 GAN_Architecture.gdraw
 gan-gen-trial
 gan-halfway-gen-transformer-d
 Generate_Examples.ipynb
 gpt-2-attempt2.ipynb
 gpt2-attempt3.ipynb
 gpt2-baseline.ipynb
 gpt2-small-rationale-generation/
 gpt2-wt-5.ipynb
 grape

In [ ]:
test_data = pd.read_csv('aita_test_set.csv')
test_data = test_data[['text', 'comments']]

In [ ]:
model_dir = "grape/hf-save-final"

# tokenizer = AutoTokenizer.from_pretrained('t5-small')
# model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512

OSError: ignored

In [ ]:
testing = test_data.iloc[400]['comments']
testing

In [ ]:
text = testing

inputs = ["summarize: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)
print(decoded_output)